In [69]:
import pandas as pd
import numpy as np
import os
import torch
import torch.nn as nn

from tools.torch_lib import *

from torch.utils.data import Dataset
from torchvision import transforms
from torch.utils.data import DataLoader
from sklearn.model_selection import train_test_split
import copy
from torchmetrics.regression import MeanAbsolutePercentageError

In [70]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [71]:
gpu = torch.device('cuda')
cpu = torch.device('cpu')
device = cpu

if torch.cuda.is_available():
    device = gpu
    # The flag below controls whether to allow TF32 on matmul. This flag defaults to False
    # in PyTorch 1.12 and later.
    torch.backends.cuda.matmul.allow_tf32 = True
    # The flag below controls whether to allow TF32 on cuDNN. This flag defaults to True.
    torch.backends.cudnn.allow_tf32 = True

print(device)

cuda


### Load dataframe

In [72]:
dataset_dir = "dataset/"
dataset_file_name = "1D_3L_chart.csv"
plots_dir = "plots/"
test_plots_dir = "test_plots/"

In [73]:
df = pd.read_csv(dataset_dir + dataset_file_name, index_col=False)
df.head()

,Unnamed: 0,AO/d,ro_formation,D/d,invasion_zone_ro,rok
0,0,0.1,0.100000,1,0.1,0.997220
1,1,0.1,0.125893,1,0.1,0.997328
2,2,0.1,0.158489,1,0.1,0.997460
3,3,0.1,0.199526,1,0.1,0.997621
4,4,0.1,0.251189,1,0.1,0.997817


In [74]:
df.columns

Index(['Unnamed: 0', 'AO/d', 'ro_formation', 'D/d', 'invasion_zone_ro', 'rok'], dtype='object')

In [75]:
# print attribute's min max

In [76]:
for column in df.columns:
    print(f"{column}: min={df[column].min()} max={df[column].max()}")

Unnamed: 0: min=0 max=5438690
AO/d: min=0.1 max=1000.0
ro_formation: min=0.1 max=10000.0
D/d: min=1 max=51
invasion_zone_ro: min=0.1 max=10000.0
rok: min=0.0898074 max=37655.3


In [77]:
attributes_to_drop = ['Unnamed: 0']
df.drop(attributes_to_drop, axis=1, inplace=True)
df.head()

,AO/d,ro_formation,D/d,invasion_zone_ro,rok
0,0.1,0.100000,1,0.1,0.997220
1,0.1,0.125893,1,0.1,0.997328
2,0.1,0.158489,1,0.1,0.997460
3,0.1,0.199526,1,0.1,0.997621
4,0.1,0.251189,1,0.1,0.997817


### Add dataframe transforms

In [78]:
inputs = np.array(['AO/d', 'ro_formation', 'invasion_zone_ro', 'D/d'])
outputs = np.array(['rok']) # 'A02M01N' dropped

In [79]:
logarithmic_columns = ['ro_formation', 'invasion_zone_ro', 'AO/d']
# normalize data ('min/max' normalization):
interval_th = [-1, 1]     # normalization interval for 'th' activation function
interval_sigmoid = [0, 1] # normalization interval for 'sigmoid' activation function
normalize_interval = interval_th

attributes_transform_dict = {}
df_transformed = df.copy()

# transform output attributes:
for output_attr in outputs:
    attr_transformer = attributes_transform_dict[output_attr] = AttributeTransformer(df_transformed[output_attr].to_numpy())

    # logarithmic transform
    forward, backward = np.log, np.exp
    df_transformed[output_attr] = attr_transformer.transform(forward, backward)
    # scaling transform
    forward, backward = get_standard_scaler_transform(attr_transformer.data)
    df_transformed[output_attr] = attr_transformer.transform(forward, backward)
    # # normalize transform
    forward, backward = get_normalize_transforms(attr_transformer.data, normalize_interval)
    df_transformed[output_attr] = attr_transformer.transform(forward, backward)

# logarithm resistance:
for col in logarithmic_columns:
    if col in outputs:
        continue
    df_transformed[col] = df_transformed[col].apply(np.log)

# add normalization
for attribute in df_transformed.columns:
    if attribute in outputs:
        continue
    transform, _ = get_standard_scaler_transform(df_transformed[attribute].to_numpy())
    df_transformed[attribute] = transform(df_transformed[attribute].to_numpy())

    transform, _ = get_normalize_transforms(df_transformed[attribute].to_numpy(), normalize_interval)
    df_transformed[attribute] = transform(df_transformed[attribute].to_numpy())

df_transformed

,AO/d,ro_formation,D/d,invasion_zone_ro,rok
0,-1.690309,-1.698416,-1.698416,-1.698416,-0.628110
1,-1.690309,-1.630478,-1.698416,-1.698416,-0.628093
2,-1.690309,-1.562543,-1.698416,-1.698416,-0.628073
3,-1.690309,-1.494606,-1.698416,-1.698416,-0.628048
4,-1.690309,-1.426669,-1.698416,-1.698416,-0.628017
...,...,...,...,...,...
5438686,1.690309,1.426669,1.698416,1.698416,0.666086
5438687,1.690309,1.494606,1.698416,1.698416,0.702592
5438688,1.690309,1.562542,1.698416,1.698416,0.739367
5438689,1.690309,1.630479,1.698416,1.698416,0.776478


### Build Datasets and create dataloaders

In [80]:
def print_inference_statistic(attributes, df_):
    means = []
    stds = []
    mins = []
    maxes = []

    for column in attributes:
        col_data = df_[column].to_numpy()

        if column in logarithmic_columns or column in outputs:
            col_data = np.log(col_data)  # first transform - log

        # col_mean = np.mean(col_data)
        # col_std = np.std(col_data)

        # means.append(col_mean)
        # stds.append(col_std)
        #
        # col_data = (col_data - col_mean) / col_std

        mins.append(np.min(col_data))
        maxes.append(np.max(col_data))

    # print(f"means={means}")
    # print(f"stds={stds}")
    print(f"mins={mins}")
    print(f"maxes={maxes}")

In [81]:
print_inference_statistic(inputs, df)

mins=[-2.3025850929940455, -2.3025850929940455, -2.3025850929940455, 1]
maxes=[6.907755278982137, 9.210340371976184, 9.210340371976184, 51]


In [82]:
print_inference_statistic(outputs, df)

mins=[-2.4100879017239056]
maxes=[10.536228993573161]


In [83]:
class SimpleDataset(Dataset):
    def __init__(self, df_, inputs, outputs, device):
        self.df = df_
        self.inputs = torch.from_numpy(df_[inputs].to_numpy()).float().to(device)
        self.outputs = torch.from_numpy(df_[outputs].to_numpy()).float().to(device)

    def __len__(self):
        return len(self.inputs)

    def __getitem__(self, idx):
        item, label = self.inputs[idx], self.outputs[idx]

        return item, label


In [84]:
batch_size = 1000

train_df, test_df = train_test_split(df_transformed, shuffle=True, test_size=0.3)
test_df, validation_df = train_test_split(test_df, shuffle=True, test_size=0.33)

train_dataset = SimpleDataset(train_df, inputs, outputs, device)
test_dataset = SimpleDataset(test_df, inputs, outputs, device)
validation_dataset = SimpleDataset(validation_df, inputs, outputs, device)
full_dataset = SimpleDataset(df_transformed, inputs, outputs, device)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size)
validation_loader = DataLoader(validation_dataset, batch_size=batch_size, shuffle=True)
full_dataset_loader = DataLoader(full_dataset, batch_size=batch_size, shuffle=True)

### Build models

In [85]:
class WeightedMAE(nn.Module):
    def __init__(self, weights):
        super(WeightedMAE, self).__init__()
        self.mae = nn.L1Loss()
        self.weights = weights

    def forward(self, inputs, targets):
        weighted_inputs = inputs * self.weights

        return self.mae(weighted_inputs, targets)

    def to(self, device):
        super().to(device)
        self.weights = self.weights.to(device)


class LinearModel(nn.Module):
    def __init__(self, layers_dims, act_str_list, output_dim):
        super().__init__()
        layers_count = len(layers_dims)
        assert layers_count > 0

        module_list = []
        for i in range(layers_count - 1):
            module_list.append(nn.Linear(layers_dims[i], layers_dims[i + 1]))
        module_list.append(nn.Linear(layers_dims[layers_count - 1], output_dim))

        activations_list = []
        for i in range(layers_count):
            activations_list.append(activations[act_str_list[i]])

        self.linears = nn.ModuleList(module_list)
        self.activations = nn.ModuleList(activations_list)

    def forward(self, x):
        y = x

        for lin, act in zip(self.linears, self.activations):
            y = lin(y)
            y = act(y)

        return y


class LinearLNormModel(nn.Module):
    def __init__(self, layers_dims, act_str_list, output_dim):
        super().__init__()
        layers_count = len(layers_dims)
        assert layers_count > 0

        linears_list = []
        layers_norm_list = []

        for i in range(layers_count - 1):
            in_features, out_features = layers_dims[i], layers_dims[i + 1]
            linears_list.append(nn.Linear(in_features, out_features))
            layers_norm_list.append(nn.LayerNorm(out_features))
        # add last layer
        linears_list.append(nn.Linear(layers_dims[layers_count - 1], output_dim))
        layers_norm_list.append(nn.LayerNorm(output_dim))

        self.linears = nn.ModuleList(linears_list)
        self.activations = nn.ModuleList([activations[act_str_list[i]] for i in range(len(act_str_list))])
        self.layer_normalizations = nn.ModuleList(layers_norm_list)

    def forward(self, x):
        y = x

        for lin, act, norm in zip(self.linears, self.activations, self.layer_normalizations):
            y = lin(y)
            y = norm(y)
            y = act(y)

        return y


# add batch normalization
class LinearBNormModel(nn.Module):
    def __init__(self, layers_dims, act_str_list, output_dim):
        super().__init__()
        layers_count = len(layers_dims)
        assert layers_count > 0

        linears_list = []
        batch_norm_list = []

        for i in range(layers_count - 1):
            in_features, out_features = layers_dims[i], layers_dims[i + 1]
            linears_list.append(nn.Linear(in_features, out_features))
            batch_norm_list.append(nn.BatchNorm1d(out_features))

        linears_list.append(nn.Linear(layers_dims[layers_count - 1], output_dim))
        batch_norm_list.append(nn.BatchNorm1d(output_dim))

        activations_list = []
        for i in range(layers_count):
            activations_list.append(activations[act_str_list[i]])

        self.linears = nn.ModuleList(linears_list)
        self.activations = nn.ModuleList(activations_list)
        self.batch_normalizations = nn.ModuleList(batch_norm_list)

    def forward(self, x):
        y = x

        for lin, act, norm in zip(self.linears, self.activations, self.batch_normalizations):
            y = lin(y)
            y = norm(y)
            y = act(y)

        return y

### Train model

In [86]:
layers_dims = [len(inputs), 40, 150, 1500, 150, 10]
layers_count = len(layers_dims)
activations_string_list = ['leaky-relu' for i in range(layers_count)]
#activations_string_list[-1] = 'sigmoid'

linear_model = LinearModel(layers_dims, activations_string_list, len(outputs)).to(device)
#linear_bn_model = LinearBNormModel(layers_dims, activations_string_list, len(outputs)).to(device)
#linear_ln_model = LinearLNormModel(layers_dims, activations_string_list, len(outputs)).to(device)

model = linear_model
model_name = "linear_model"
linear_model

LinearModel(
  (linears): ModuleList(
    (0): Linear(in_features=4, out_features=40, bias=True)
    (1): Linear(in_features=40, out_features=150, bias=True)
    (2): Linear(in_features=150, out_features=1500, bias=True)
    (3): Linear(in_features=1500, out_features=150, bias=True)
    (4): Linear(in_features=150, out_features=10, bias=True)
    (5): Linear(in_features=10, out_features=1, bias=True)
  )
  (activations): ModuleList(
    (0-5): 6 x LeakyReLU(negative_slope=0.01)
  )
)

In [87]:
learning_rate = 0.0001
epoch_count = 350

optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

#loss_function = WeightedMAE(torch.tensor([1.0, 1.0, 1.0, 1.0, 1.0], dtype=float))
loss_function = nn.L1Loss()

In [88]:
epoch_validation = True
train_loss_threshold = 0.0003

train_loss_list, validation_loss_list = train_model(epoch_count, model, optimizer, loss_function, train_loader, validation_loader, True, train_loss_threshold)
plot_loss(train_loss_list, "train loss")

Epoch: 0; train loss=0.071461; validation loss=0.020676
Epoch: 1; train loss=0.013553; validation loss=0.011162
Epoch: 2; train loss=0.009789; validation loss=0.009216
Epoch: 3; train loss=0.008194; validation loss=0.007427
Epoch: 4; train loss=0.007304; validation loss=0.010042
Epoch: 5; train loss=0.006675; validation loss=0.007145
Epoch: 6; train loss=0.006177; validation loss=0.005659
Epoch: 7; train loss=0.005831; validation loss=0.005562
Epoch: 8; train loss=0.005502; validation loss=0.005046
Epoch: 9; train loss=0.005181; validation loss=0.004873
Epoch: 10; train loss=0.004942; validation loss=0.005154
Epoch: 11; train loss=0.004718; validation loss=0.004481
Epoch: 12; train loss=0.004537; validation loss=0.004241
Epoch: 13; train loss=0.004374; validation loss=0.004894
Epoch: 14; train loss=0.004271; validation loss=0.004161
Epoch: 15; train loss=0.004088; validation loss=0.004084
Epoch: 16; train loss=0.003952; validation loss=0.004775
Epoch: 17; train loss=0.003884; validatio

KeyboardInterrupt: 

In [ ]:
test_loss = test_loop(test_loader, model, loss_function)
print(f"test loss={test_loss}")

In [ ]:
plot_loss(validation_loss_list, "test loss")

### Plot predictions

In [ ]:
for _, (X, y) in enumerate(train_loader):
    print(model(X))
    break

In [ ]:
# plot_predictions(outputs, full_dataset_loader, linear_model)

In [ ]:
#plot_actual_predictions(outputs, full_inference_dataset_loader, linear_model, attributes_transform_dict, df)

In [ ]:
plot_relative_errors(outputs, full_dataset_loader, model, attributes_transform_dict,
                     df, 0.01, device, plots_dir, mode='default+hist', bin_count=100)

In [ ]:
# plot test predictions:
plot_relative_errors(outputs, test_loader, model, attributes_transform_dict,
                     df, 0.01, device, plots_dir + test_plots_dir, mode='default+hist', bin_count=100)

#### check predictions manually

In [ ]:
predictor = Predictor(full_dataset_loader, df, attributes_transform_dict, model, inputs, outputs)
predictions_dict, actuals_dict = predictor.predict(device)

In [ ]:
def compare_prediction(idx: int, prediction_dict, actuals_dict, attribute):
    predicted = prediction_dict[attribute][idx]
    actual = actuals_dict[attribute][idx]
    relative_error = abs(actual - predicted) / actual
    print(f"{idx}: predicted={predicted}; actual={actual}; relative error={relative_error}")

### Save model

In [ ]:
model.to(cpu)    # attach model to cpu before scripting and saving to prevent cuda meta information saved
scripted_model = torch.jit.script(model)
model_file_name = "saved_models/" + model_name + str(round(test_loss, 7)).replace('.', '_')

scripted_model.save(model_file_name + ".pt") # save torch script model which compatible with pytorch c++ api
torch.save(model, model_file_name + ".pth")   # save model in python services specific format

# attach model back to device:
model.to(device)

In [ ]:
scripted_model(torch.tensor([0.6, 0.362372, 0.04]))

In [ ]:
model(torch.tensor([0.6, 0.362372, 0.04], device=device))